We're going to start by grabbing the geometry for the Austin community area.

In [17]:
import requests
from shapely.geometry import shape, Point

r = requests.get('https://data.cityofchicago.org/api/geospatial/cauq-8yn6?method=export&format=GeoJSON')

for feature in r.json()['features']:
    if feature['properties']['community'] == 'AUSTIN':
        austin = feature

poly = shape(austin['geometry'])

Now let's get the shootings data.

In [15]:
NDB_URL = 'http://newsroomdb.tribapps.com/table/json/%s'

def get_data(table):
    r = requests.get(NDB_URL % table)
    return r.json()

shootings = get_data('shootings')

Now let's iterate through the shootings, generate shapely points and check to see if they're in the geometry we care about.

In [22]:
found = []

for row in shootings:
    if not row['Geocode Override']:
        continue
    points = row['Geocode Override'][1:-1].split(',')
    if len(points) != 2:
        continue
    point = Point(float(points[1]), float(points[0]))
    if poly.contains(point):
        found.append(row)

print 'Found %d shootings in this community area' % len(found)
for f in found:
    print f['Date'], f['Age'], f['Sex'], f['Shooting Location']

Found 882 shootings in this community area
2011-09-25 22 M 1300 N. Mayfield Avenue, Chicago, IL
2011-12-18 19 F 1159 N. Parkside Avenue, Chicago, IL
2011-12-21 46 M 101 N. Pine Avenue, Chicago, IL
2011-12-24 20 M 5501 W. Bloomingdale Avenue, Chicago, IL
2011-12-29 41 M 1201 N. Austin Boulevard, Chicago, IL
2012-01-31 17 M 5846 W. Augusta Boulevard, Chicago, IL
2012-03-03 20 M 5331 W. Congress Parkway, Chicago, IL
2012-03-11 15 M 958 N. Massasoit Avenue, Chicago, IL
2012-03-11 14 M 958 N. Massasoit Avenue, Chicago, IL
2012-03-15 21 M 4945 W. Ferdinand Street, Chicago, IL
2012-03-15 15 M 5522 W. Monroe Street, Chicago, IL
2012-03-25 22 M 1501 N. Lockwood Avenue, Chicago, IL
2012-06-01 30 M 5045 W. Congress Parkway, Chicago, IL
2012-06-02 28 M 5301 W. Monroe Street, Chicago, IL
2012-06-03 20 M 5733 W. Bloomingdale Avenue, Chicago, IL
2012-06-05 22 M 1055 N. Long Avenue, Chicago, IL
2012-06-15 16 M 1035 N. Long Avenue, Chicago, IL
2012-06-19 25 M 1217 N. Mason Avenue, Chicago, IL
2012-06-2